In [1]:
# 5/26
import pandas as pd
import numpy as np
# 파일 저장및 불러오기 모듈
import bz2
import pickle
import _pickle as cPickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = cPickle.load(data)
    return data
# 불러온 데이터는 전처리 완료된 데이터들이 분할된 형태
data1 = decompress_pickle('texi_train/train1.pbz2')
data2 = decompress_pickle('texi_train/train2.pbz2')
data3 = decompress_pickle('texi_train/train3.pbz2')
df_list = []
df_list.append(data1)
df_list.append(data2)
df_list.append(data3)

In [3]:
texi_df_all = pd.concat(df_list, ignore_index=True)
texi_df_all.sample(3)

,index,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,day,hour,distance
15130511,46588671,6.5,2012-08-03 00:51:00+00:00,-74.000940,40.731625,-74.002070,40.750732,2012,8,3,0,0.019140
17625668,54270369,11.0,2013-10-18 23:10:00+00:00,-74.009640,40.704906,-73.995350,40.728268,2013,10,18,23,0.027386
49153919,40498124,15.5,2015-05-16 19:36:00+00:00,-73.978302,40.736675,-73.995789,40.723389,2015,5,16,19,0.021962


In [4]:
del data1, data2, data3, df_list

In [5]:
#데이터를 다 사용하면 컴퓨터가 버티질 못한다 여기서 최대한 골고루 들어가게 10분의 1로 데이터 추출해서 활용하자
texi_df = texi_df_all.sample(5400000, random_state=15)

In [77]:
texi_df

,index,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,day,hour,distance
16973830,52263852,10.9,2011-01-13 11:20:00+00:00,-73.957230,40.768910,-73.977330,40.757120,2011,1,13,11,0.023303
50034343,43209749,10.5,2009-12-02 08:34:00+00:00,-73.982773,40.721264,-74.006119,40.705658,2009,12,2,8,0.028082
21507333,10798948,8.5,2010-08-19 09:59:00+00:00,-73.991150,40.755035,-73.995796,40.743389,2010,8,19,9,0.012539
21340781,10285876,17.0,2012-09-22 21:41:00+00:00,-73.955482,40.765823,-74.003212,40.732883,2012,9,22,21,0.057993
22732867,14570581,4.5,2011-07-31 21:35:00+00:00,-73.937157,40.797829,-73.948235,40.797256,2011,7,31,21,0.011093
...,...,...,...,...,...,...,...,...,...,...,...,...
35781699,54747850,14.5,2010-05-08 00:31:00+00:00,-73.991417,40.685246,-73.985046,40.719112,2010,5,8,0,0.034461
24537187,20126125,7.5,2013-09-29 10:12:00+00:00,-73.988068,40.774788,-73.964958,40.769314,2013,9,29,10,0.023749
34836143,51836212,6.5,2011-01-07 15:31:00+00:00,-73.961243,40.765144,-73.975494,40.760227,2011,1,7,15,0.015076
40436048,13654877,13.5,2014-02-12 12:13:00+00:00,-73.789612,40.647606,-73.771423,40.659283,2014,2,12,12,0.021614


In [6]:
# 2009년 1월 1일 은 목요일
# 2012 년은 윤년
def outputdate(date):
        year = int(date[:4])
        month = int(date[5:7])
        day = int(date[8:10])        
        m = [1, 2, 3 ,4 ,5 ,6 ,7, 8, 9, 10, 11, 12]
        li = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        sum_day = 0       
        t= 0    
        if year == 2009:
            for k in li[:(month-1)]:
                sum_day +=k                
            sum_day += day
            t = sum_day % 7             
        elif year == 2010:
            sum_day = 365
            for k in li[:(month-1)]:
                sum_day +=k
            sum_day += day
            t = sum_day % 7
        elif year == 2011:
            sum_day = 365+365
            for k in li[:(month-1)]:
                sum_day +=k
            sum_day += day
            t = sum_day % 7
        elif year == 2012:
            sum_day = 365+365+365
            li[1] = 28
            for k in li[:(month-1)]:
                sum_day +=k
            sum_day += day
            t = sum_day % 7
        elif year == 2013:
            li[1]= 29
            sum_day = 365+365+365+366
            for k in li[:(month-1)]:
                sum_day +=k
            sum_day += day
            t = sum_day % 7
        elif year == 2014:
            sum_day = 365+365+365+366+365
            for k in li[:(month-1)]:
                sum_day +=k
            sum_day += day
            t = sum_day % 7
        elif year == 2015:
            sum_day = 365+365+365+366+365+365
            for k in li[:(month-1)]:
                sum_day +=k
            sum_day += day
            t = sum_day % 7
        if t==0: 
            return 3
        elif t==1: 
            return 4
        elif t==2:
            return 5
        elif t==3:
            return 6
        elif t==4:
            return 7
        elif t==5:
            return 1
        elif t==6:
            return 2

In [7]:
texi_df['pickup_datetime'] = texi_df['pickup_datetime'].astype('str')

In [8]:
texi_df["date"]=texi_df['pickup_datetime'].apply(lambda x: outputdate(x))

In [9]:
texi_df 

,index,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,day,hour,distance,date
16973830,52263852,10.9,2011-01-13 11:20:00+00:00,-73.957230,40.768910,-73.977330,40.757120,2011,1,13,11,0.023303,4
50034343,43209749,10.5,2009-12-02 08:34:00+00:00,-73.982773,40.721264,-74.006119,40.705658,2009,12,2,8,0.028082,4
21507333,10798948,8.5,2010-08-19 09:59:00+00:00,-73.991150,40.755035,-73.995796,40.743389,2010,8,19,9,0.012539,5
21340781,10285876,17.0,2012-09-22 21:41:00+00:00,-73.955482,40.765823,-74.003212,40.732883,2012,9,22,21,0.057993,5
22732867,14570581,4.5,2011-07-31 21:35:00+00:00,-73.937157,40.797829,-73.948235,40.797256,2011,7,31,21,0.011093,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35781699,54747850,14.5,2010-05-08 00:31:00+00:00,-73.991417,40.685246,-73.985046,40.719112,2010,5,8,0,0.034461,7
24537187,20126125,7.5,2013-09-29 10:12:00+00:00,-73.988068,40.774788,-73.964958,40.769314,2013,9,29,10,0.023749,1
34836143,51836212,6.5,2011-01-07 15:31:00+00:00,-73.961243,40.765144,-73.975494,40.760227,2011,1,7,15,0.015076,5
40436048,13654877,13.5,2014-02-12 12:13:00+00:00,-73.789612,40.647606,-73.771423,40.659283,2014,2,12,12,0.021614,3


In [31]:
# 압축 저장
def compressed_pickle(title, data):
    with bz2.BZ2File(title+'.pbz2', 'w') as f:
        cPickle.dump(data, f)
# 파일명과, 저장을 원하는 데이터
compressed_pickle('texi_train/adddate', texi_df)

In [10]:
texi_df.date.value_counts()

5    816966
6    816129
4    801287
7    779162
3    773340
2    712336
1    700780
Name: date, dtype: int64

In [15]:
texi_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5400000 entries, 16973830 to 9199245
Data columns (total 13 columns):
 #   Column             Dtype  
---  ------             -----  
 0   index              int64  
 1   fare_amount        float64
 2   pickup_datetime    object 
 3   pickup_longitude   float64
 4   pickup_latitude    float64
 5   dropoff_longitude  float64
 6   dropoff_latitude   float64
 7   year               int64  
 8   month              int64  
 9   day                int64  
 10  hour               int64  
 11  distance           float64
 12  date               int64  
dtypes: float64(6), int64(6), object(1)
memory usage: 576.8+ MB


In [20]:
texi_df["year"] = texi_df['year'].astype("str")
texi_df["month"] = texi_df['month'].astype("str")

In [22]:
texi_df["hour"] = texi_df['hour'].astype("str")
texi_df["date"] = texi_df['date'].astype("str")

In [23]:
a = pd.get_dummies(texi_df[["year",'month','hour','date']])

In [25]:
a

,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,month_1,month_10,month_11,...,hour_7,hour_8,hour_9,date_1,date_2,date_3,date_4,date_5,date_6,date_7
16973830,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
50034343,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
21507333,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
21340781,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
22732867,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35781699,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
24537187,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
34836143,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
40436048,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [24]:
x_data =texi_df[["pickup_longitude","pickup_latitude", "dropoff_longitude", "dropoff_latitude", "distance"]]
y_target = texi_df["fare_amount"]

In [26]:
a[["pickup_longitude","pickup_latitude", "dropoff_longitude", "dropoff_latitude", "distance"]] = texi_df[["pickup_longitude","pickup_latitude", "dropoff_longitude", "dropoff_latitude", "distance"]]
a

,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,month_1,month_10,month_11,...,date_3,date_4,date_5,date_6,date_7,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,distance
16973830,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,-73.957230,40.768910,-73.977330,40.757120,0.023303
50034343,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,-73.982773,40.721264,-74.006119,40.705658,0.028082
21507333,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,-73.991150,40.755035,-73.995796,40.743389,0.012539
21340781,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,-73.955482,40.765823,-74.003212,40.732883,0.057993
22732867,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,-73.937157,40.797829,-73.948235,40.797256,0.011093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35781699,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,-73.991417,40.685246,-73.985046,40.719112,0.034461
24537187,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,-73.988068,40.774788,-73.964958,40.769314,0.023749
34836143,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,-73.961243,40.765144,-73.975494,40.760227,0.015076
40436048,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,-73.789612,40.647606,-73.771423,40.659283,0.021614


In [27]:
x_data = a

In [8]:
x_data

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year,month,hour,distance
11234083,-73.979370,40.751812,-73.987000,40.749530,2014,7,21,0.007964
43243411,-73.963173,40.772671,-73.971390,40.743244,2012,2,17,0.030552
52993808,-73.957535,40.767380,-73.956001,40.779785,2009,3,17,0.012500
3412680,-73.790310,40.647083,-73.979380,40.578320,2012,1,20,0.201186
48919756,-73.997131,40.747124,-73.986900,40.761108,2010,2,0,0.017328
...,...,...,...,...,...,...,...,...
1162876,-73.782000,40.644720,-73.897420,40.748436,2011,8,17,0.155173
37044091,-73.956604,40.783916,-73.980827,40.763676,2012,2,8,0.031567
20524919,-73.989906,40.733624,-73.997780,40.604218,2015,5,1,0.129645
13496589,-73.964035,40.803123,-73.959694,40.810390,2014,2,3,0.008465


In [9]:
y_data

11234083     5.5
43243411    12.1
52993808     5.7
3412680     34.5
48919756     5.3
            ... 
1162876     33.3
37044091     7.7
20524919    34.0
13496589     4.0
50150527     7.5
Name: fare_amount, Length: 5400000, dtype: float64

In [29]:
x_tr, x_te, y_tr, y_te = train_test_split(x_data, y_target, test_size=0.2, random_state=5)
lr_clf = LinearRegression()
lr_clf.fit(x_tr, y_tr)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [30]:
lr_pre = lr_clf.predict(x_te)
mse = mean_squared_error(y_te, lr_pre)
rmse = np.sqrt(mse)
print("MSE : {0:.3f}, RMSE : {1:.3f}".format(mse,rmse))
print("Variance score : {0:.3f}".format(r2_score(y_te, lr_pre)))

MSE : 19.155, RMSE : 4.377
Variance score : 0.786


In [32]:
print("절편 값")
np.round(lr_clf.intercept_,2)

절편 값


356573810620.33

In [33]:
print("각 컬럼별 회귀 계수 값")
np.round(lr_clf.coef_,2)

각 컬럼별 회귀 계수 값


array([ 7.96331184e+08,  7.96331184e+08,  7.96331184e+08,  7.96331184e+08,
        7.96331186e+08,  7.96331186e+08,  7.96331186e+08, -1.20572609e+08,
       -1.20572609e+08, -1.20572609e+08, -1.20572608e+08, -1.20572609e+08,
       -1.20572609e+08, -1.20572609e+08, -1.20572609e+08, -1.20572609e+08,
       -1.20572609e+08, -1.20572609e+08, -1.20572608e+08,  2.98720286e+10,
        2.98720286e+10,  2.98720286e+10,  2.98720286e+10,  2.98720286e+10,
        2.98720286e+10,  2.98720286e+10,  2.98720286e+10,  2.98720286e+10,
        2.98720286e+10,  2.98720286e+10,  2.98720286e+10,  2.98720286e+10,
        2.98720286e+10,  2.98720286e+10,  2.98720286e+10,  2.98720286e+10,
        2.98720286e+10,  2.98720286e+10,  2.98720286e+10,  2.98720286e+10,
        2.98720286e+10,  2.98720286e+10,  2.98720286e+10, -3.87121597e+11,
       -3.87121597e+11, -3.87121597e+11, -3.87121597e+11, -3.87121597e+11,
       -3.87121597e+11, -3.87121597e+11,  8.28000000e+00,  4.30000000e+00,
       -7.29000000e+00, -

In [34]:
y_te

40709280     6.5
40959768    21.0
37398280     3.0
6874465     12.0
7709664      6.1
            ... 
40481277    11.3
19327472    11.3
51296640    17.0
32977687     9.3
43262466     7.7
Name: fare_amount, Length: 1080000, dtype: float64

In [35]:
result_df = pd.DataFrame(y_te.values, columns=['y_t'])
result_df["pred"]= np.round(lr_pre, 1)
result_df["diff"]= np.abs(result_df["y_t"]- result_df["pred"])
result_df

,y_t,pred,diff
0,6.5,8.7,2.2
1,21.0,13.8,7.2
2,3.0,5.0,2.0
3,12.0,14.3,2.3
4,6.1,7.8,1.7
...,...,...,...
1079995,11.3,12.4,1.1
1079996,11.3,13.2,1.9
1079997,17.0,12.1,4.9
1079998,9.3,3.4,5.9


In [36]:
result_df["diff"].mean()

2.147045456990186

In [37]:
import seaborn as sns
import matplotlib.pyplot as plt

In [38]:
# sns.distplot(y_te, y_te.reset_index().index)

In [39]:
from sklearn.model_selection import cross_val_score
# cross_val_score()로 5Fold 셋으로 실행
neg_mse_scores = cross_val_score(lr_clf, x_data, y_target, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1*neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

#cross_val_score(scoring="neg_mean_squared_error") 로 반환된 값은 모두 음수
print("5 folds 의 개별 평균 Negative MSE scores: ", np.round(neg_mse_scores, 2))
print("5 folds 의 개별 RMSE scores", np.round(rmse_scores, 2))
print("5 folds 의 평균 RMSE : {0:.3f}".format(avg_rmse) )

5 folds 의 개별 평균 Negative MSE scores:  [-19.17 -20.46 -19.44 -19.7  -99.24]
5 folds 의 개별 RMSE scores [4.38 4.52 4.41 4.44 9.96]
5 folds 의 평균 RMSE : 5.542


In [40]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_target, test_size=0.2, random_state=1)

lr_reg = LinearRegression()
lr_reg.fit(x_train, y_train)
pred = lr_reg.predict(x_test)
print(pred)

[ 7.60015869 10.34307861 10.99713135 ...  8.95758057  8.02172852
  5.83807373]


In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
mae_val = mean_absolute_error(y_test, pred)
print("MAE: {0:.3F}".format(mae_val))
mse_val = mean_squared_error(y_test, pred)
print("MSE: {0:.3F}".format(mse_val))
rmse_val = np.sqrt(mse_val)
print("RMSE: {0: .3F}".format(rmse_val))


MAE: 2.151
MSE: 19.779
RMSE:  4.447


In [ ]:
========================================================여기까지..하고 다음 노트북에 이어서

In [56]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
degrees = [1,2, 3 ,4, 5, 9]

for i in range(len(degrees)):   
    
    polynomial_features = PolynomialFeatures(degree = degrees[i], include_bias=False)
    lr =LinearRegression()
    pipeline = Pipeline([("poly",polynomial_features),("lr",lr)])
    pipeline.fit(x.reshape(-1, 1), y)
    
    # 교차 검증으로 다항 회귀를 평가
    scores = cross_val_score(pipeline, x.reshape(-1,1), y, scoring="neg_mean_squared_error", cv=10)
    coe = pipeline.named_steps['lr'].coef_
    print("Degree {0} 회귀 계수는 {1} ".format(degrees[i], np.round(coe,4)))
    print("MSE 는 : {0}".format(-1*np.mean(scores)))
    
   

In [ ]:
 Regularized Linear Models - Ridge, Lasso
    Ridge Regression
    : L2 규제 로 w의 제곱 값에 규제

In [ ]:
ridge = Ridge(alpha = 10)
# 교차 검증인데;;
neg_mse_score = cross_val_score(ridge, x_data, y_target, scoring="neg_mean_squared_error", cv=5)
rmse_score = np.sqrt(-1*neg_mse_score)
avg_rmse = np.mean(rmse_score)
print("5 folds의 개별 Negative MSE score:", np.round(neg_mse_score,3))
print("5 folds의 개별 RMSE score:", np.round(rmse_score,3))
print("5 folds의 평균 RMSE score:", np.round(avg_rmse, 3))

In [ ]:
alphas = [0, 0.1, 1, 10, 100, 120, 140]
for alpha in alphas:
    ridge = Ridge(alpha = alpha)
    # 교차 검증인데;;
    neg_mse_score = cross_val_score(ridge, x_data, y_target, scoring="neg_mean_squared_error", cv=5)
    rmse_score = np.sqrt(-1*neg_mse_score)
    avg_rmse = np.mean(rmse_score)
    print("alpha {0} 의 5 folds의 평균 RMSE score:".format(alpha), np.round(avg_rmse, 3))

In [ ]:
 라쏘 회귀

In [ ]:
from sklearn.linear_model import Lasso, ElasticNet

In [ ]:
# 곅
def get_linear_reg_eval (model_name, params=None, x_data_n=None, y_target_n = None, verbose=True):
    coeff_df = pd.DataFrame()
    print("####",model_name,"####")
    # alpha 값을 바꿔가며 반복
    for param in params:
        if model_name =='Ridge': model = Ridge(alpha=param)
        elif model_name =='Lasso': model = Lasso(alpha=param)
        elif model_name =='ElasticNet': model = ElasticNet(alpha=param, l1_ratio=0.7)
          
        neg_mse_scores = cross_val_score(model, x_data_n, y_target_n, scoring="neg_mean_squared_error", cv=5)
        avg_rmse = np.mean(np.sqrt(-1*neg_mse_scores))
        print("alpha {0} 의 5 folds의 평균 RMSE score:{1:.3f}".format(param, avg_rmse))
        # 모델 학습 시키기
        model.fit(x_data_n, y_target_n)
        coeff = pd.Series(data=model.coef_, index=x_data_n.columns)
        colname = 'alpha:'+str(param)
        coeff_df[colname] = coeff
    return coeff_df
          

In [ ]:
lasso_alphas = [0.07, 0.1, 0.5,1,3]
coe_lasso_df = get_linear_reg_eval('Lasso', params=lasso_alphas, x_data_n = x_data, y_target_n=y_target)

In [ ]:
 엘라스틱넷 회귀

In [ ]:
elastic_alphas = [0.07, 0.1, 0.5, 1, 3]
coeff_elastic_df = get_linear_reg_eval('ElasticNet', params=elastic_alphas, x_data_n = x_data, y_target_n=y_target)